<a href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/Real_CHEMBERT_Validation_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Running the baseline**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/baseline.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>  

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [ ]:
!git clone https://github.com/raphaelrubrice/BornToOverfit.git
%cd BornToOverfit
!git checkout Camille

Cloning into 'BornToOverfit'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 550 (delta 80), reused 78 (delta 42), pack-reused 419 (from 1)
Receiving objects: 100% (550/550), 392.19 KiB | 17.05 MiB/s, done.
Resolving deltas: 100% (359/359), done.
/content/BornToOverfit
Branch 'Camille' set up to track remote branch 'Camille' from 'origin'.
Switched to a new branch 'Camille'


Installing requirements

In [ ]:
!pip install -r data_baseline/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 76.9 MB/s eta 0:00:00


**Downloading the data**

In [ ]:
!cp -r /content/drive/MyDrive/Kaggle_ALTEGRAD/data /content/BornToOverfit/data_baseline/.

cp: cannot open '/content/drive/MyDrive/Kaggle_ALTEGRAD/data/results/baseline.gsheet' for reading: Operation not supported


**Génération / chargement des embedding pour le modèle souhaité (BERT, ChemBERT etc)**


In [ ]:
!python data_baseline/generate_description_embeddings_automatic.py \
    --data_dir data_baseline/data \
    --model_name recobo/chemical-bert-uncased \
    --pooling cls \
    --splits train validation

Loading model: recobo/chemical-bert-uncased
tokenizer_config.json: 100% 339/339 [00:00<00:00, 2.69MB/s]
config.json: 100% 612/612 [00:00<00:00, 5.46MB/s]
vocab.txt: 228kB [00:00, 11.4MB/s]
special_tokens_map.json: 100% 112/112 [00:00<00:00, 1.08MB/s]
2026-01-09 10:38:03.484041: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-09 10:38:03.501638: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767955083.523552    5524 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767955083.530174    5524 cuda_blas.cc:1407] Unable to reg

In [ ]:
%%bash
DRIVE_DIR="/content/drive/MyDrive/Kaggle_ALTEGRAD/data"

echo "📂 Copie des fichiers vers le Drive..."
mkdir -p "$DRIVE_DIR" # Crée le dossier s'il n'existe pas

cp data_baseline/data/train_embeddings_RealChemBERT.pt "$DRIVE_DIR/"
cp data_baseline/data/validation_embeddings_RealChemBERT.pt "$DRIVE_DIR/"

echo "✅ Fichiers sauvegardés en sécurité sur le Drive !"
ls -lh "$DRIVE_DIR/train_embeddings_RealChemBERT.pt"

📂 Copie des fichiers vers le Drive...
✅ Fichiers sauvegardés en sécurité sur le Drive !
-rw------- 1 root root 92M Jan  9 10:44 /content/drive/MyDrive/Kaggle_ALTEGRAD/data/train_embeddings_RealChemBERT.pt


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import subprocess
from pathlib import Path

# 1. Configuration des chemins
# Assurez-vous que ce dossier correspond bien à là où sont vos fichiers CSV
DATA_DIR = Path("data_baseline/data")
TRAIN_FILE = DATA_DIR / "train_embeddings.csv"
VAL_FILE = DATA_DIR / "validation_embeddings.csv"

# Fonction pour charger et convertir le CSV (crucial !)
def load_embeddings_csv(filepath):
    print(f"📥 Chargement de {filepath}...")
    df = pd.read_csv(filepath)

    # La partie magique : convertir la chaîne "0.1, 0.2, ..." en tableau de nombres
    # On utilise tqdm si vous l'avez, sinon on fait une liste simple
    try:
        from tqdm import tqdm
        iterator = tqdm(df['embedding'], desc="Conversion str->num")
    except ImportError:
        iterator = df['embedding']

    # Conversion de la colonne texte en liste de floats
    embeddings_list = [np.fromstring(x, sep=',') for x in iterator]

    # Création du Tensor
    embeddings_tensor = torch.tensor(np.array(embeddings_list), dtype=torch.float32)
    ids = df['ID'].tolist()

    print(f"✅ Chargé ! Shape: {embeddings_tensor.shape}")
    return ids, embeddings_tensor

# 2. Logique Principale
if TRAIN_FILE.exists() and VAL_FILE.exists():
    print("🚀 Fichiers trouvés ! Chargement direct sans recalculer...")

    train_ids, train_embeddings = load_embeddings_csv(TRAIN_FILE)
    val_ids, val_embeddings = load_embeddings_csv(VAL_FILE)

else:
    print("⚠️ Fichiers non trouvés (ou incomplets).")
    print("⚙️ Lancement du script de génération...")

    # Commande à exécuter
    cmd = [
        "python", "data_baseline/generate_description_embeddings_automatic.py",
        "--data_dir", "data_baseline/data",
        "--model_name", "recobo/chemical-bert-uncased",
        "--pooling", "cls",
        "--splits", "train", "validation"
    ]

    # Exécution de la commande bash depuis Python
    process = subprocess.run(cmd, capture_output=False) # capture_output=False affiche les logs en direct

    if process.returncode == 0:
        print("\n✅ Génération terminée avec succès !")
        # Note : Le script de génération aura probablement créé des fichiers nommés
        # 'train_embeddings.csv' (sans _ChemBERT). Pensez à vérifier les noms exacts
        # si vous devez les recharger juste après.
    else:
        print("\n❌ Erreur lors de la génération.")

🚀 Fichiers trouvés ! Chargement direct sans recalculer...
📥 Chargement de data_baseline/data/train_embeddings.csv...


Conversion str->num: 100%|██████████| 31008/31008 [00:04<00:00, 7674.15it/s]


✅ Chargé ! Shape: torch.Size([31008, 768])
📥 Chargement de data_baseline/data/validation_embeddings.csv...


Conversion str->num: 100%|██████████| 1000/1000 [00:00<00:00, 7845.44it/s]

✅ Chargé ! Shape: torch.Size([1000, 768])


## **Training**

In [ ]:
!mkdir -p results

# 2. Lancer l'entraînement
!python data_baseline/train_gcn_v3_gps_PT_args_pooling.py \
  --loss, loss_type \
  --pooling, 'sum' \
  --batch-size, 128 \
  --epochs, 150 \
  --data_dir data_baseline/data \
  --train_emb data_baseline/data/train_embeddings_RealChemBERT.pt \
  --val_emb data_baseline/data/validation_embeddings_RealChemBERT.pt \
  --out_ckpt results/model_gps_Realchembert_mse.pt



#'python', '-u', str(SCRIPT_PATH),
#                '--loss', loss_type,
#                '--pooling', config['pooling'],
#                '--batch-size', str(BATCH_SIZE),
#                '--epochs', '150',
#                '--lr', LR,
#                '--data-dir', DATA_DIR_ARG,
#                '--hidden-dim', str(config['hidden_dim']),
#                '--num-layers', str(config['num_layers']),
#                '--dropout', str(config['dropout']),
#                '--num-heads', str(config['num_heads'])

GPS + ChemicalBERT + MSE Loss
Device: cuda
⚡ Chargement rapide depuis data_baseline/data/train_embeddings_RealChemBERT.pt...
⚡ Chargement rapide depuis data_baseline/data/validation_embeddings_RealChemBERT.pt...
Embedding dimension: 768
Loading graphs from: data_baseline/data/train_graphs.pkl
Loaded 31008 graphs
Loading graphs from: data_baseline/data/validation_graphs.pkl
Loaded 1000 graphs
Epoch 1/50 | loss=0.000189 | {'MRR': 0.14092133939266205, 'R@1': 0.05300000309944153, 'R@5': 0.2110000103712082, 'R@10': 0.33800002932548523}
Epoch 1/50 | loss=0.000189 | {'MRR': 0.14092133939266205, 'R@1': 0.05300000309944153, 'R@5': 0.2110000103712082, 'R@10': 0.33800002932548523}
  → New best MRR: 0.1409 | saved: results/model_gps_Realchembert_mse.pt
Loading graphs from: data_baseline/data/validation_graphs.pkl
Loaded 1000 graphs
Epoch 2/50 | loss=0.000139 | {'MRR': 0.22109456360340118, 'R@1': 0.11100000888109207, 'R@5': 0.320000022649765, 'R@10': 0.45000001788139343}
  → New best MRR: 0.2211 | 

## **Results - Retrieval**

In [ ]:
import os
from pathlib import Path

# --- VOS VARIABLES À MODIFIER ICI ---
CODE_NAME   = "train_gps_chembert_mse"      # Nom du fichier .py (sans .py)
MODEL_FILE  = "model_gps_Realchembert_mse.pt"   # Nom du fichier modèle
DATA_DIR    = "data_baseline/data"              # Dossier des données
RESULTS_DIR = "results"                         # Dossier où le modèle est sauvegardé
SUFFIX      = "_50epochs"                       # Suffixe pour la copie Drive

# Chemins absolus
BASE_PATH  = Path("/content/BornToOverfit")
DATA_PATH  = BASE_PATH / DATA_DIR
MODEL_PATH = BASE_PATH / RESULTS_DIR / MODEL_FILE
DRIVE_PATH = Path("/content/drive/MyDrive/Kaggle_ALTEGRAD/data/model")

print(f"🔹 Code utilisé : {CODE_NAME}.py")
print(f"🔹 Modèle cible : {MODEL_PATH}")
print(f"🔹 Suffixe copie : {SUFFIX}")

# ==============================================================================
# CORRECTIF IMPORTANT : EXPORT VERS L'ENVIRONNEMENT BASH
# ==============================================================================
# Sans ces lignes, la cellule %%bash ne voit pas les variables
os.environ['CODE_NAME']   = CODE_NAME
os.environ['MODEL_FILE']  = MODEL_FILE
os.environ['DATA_DIR']    = DATA_DIR
os.environ['RESULTS_DIR'] = RESULTS_DIR
os.environ['MODEL_PATH']  = str(MODEL_PATH)
os.environ['DRIVE_PATH']  = str(DRIVE_PATH)
os.environ['SUFFIX']      = SUFFIX

🔹 Code utilisé : train_gps_chembert_mse.py
🔹 Modèle cible : /content/BornToOverfit/results/model_gps_Realchembert_mse.pt
🔹 Suffixe copie : _50epochs


In [ ]:
# Lancer le script de retrieval avec les variables
%%bash
echo "🚀 Lancement du retrieval..."
python data_baseline/retrieval_answer_new.py \
  --code $CODE_NAME \
  --model $MODEL_FILE \
  --data_dir $DATA_DIR \
  --results_dir $RESULTS_DIR

# Copie vers le Drive avec le SUFFIXE variable
echo -e "\n💾 Copie vers le Drive..."
mkdir -p "$DRIVE_PATH"

# Construction du nouveau nom : NomOriginal (sans extension) + Suffixe + .pt
cp "$MODEL_PATH" "$DRIVE_PATH/${MODEL_FILE%.*}${SUFFIX}.pt"

echo "✅ Copié vers : $DRIVE_PATH/${MODEL_FILE%.*}${SUFFIX}.pt"

🚀 Lancement du retrieval...
🔄 Import dynamique du module : train_gps_chembert_mse ...
Device: cuda
Loading embeddings from data_baseline/data/train_embeddings_ChemBERT.csv...
Initializing model (dim=768)...
Loading weights from results/model_gps_Realchembert_mse.pt...
Loading descriptions from data_baseline/data/train_graphs.pkl...
Train set size: 31008
Loading graphs from: data_baseline/data/test_graphs.pkl
Loaded 1000 graphs
Test set size: 1000
Encoded 1000 test molecules

[Test ID 0] -> Closest Train ID: 13434
Desc: The molecule is a cholesterol ester obtained by the formal condensation of cholesterol with palmitol...

[Test ID 1] -> Closest Train ID: 16242
Desc: The molecule is a tripeptide composed of L-aspartic acid, glycine, and L-proline units joined in seq...

[Test ID 2] -> Closest Train ID: 30518
Desc: The molecule is an oxysterol that is the 22S-hydroxy derivative of cholesterol. It is a 22-hydroxy s...

Saved 1000 retrieved descriptions to: data_baseline/data/test_retrieve

## **Evaluation**

In [ ]:
!python -m pip -q install -U evaluate bert-score sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.7 MB/s eta 0:00:00


In [ ]:
import pickle
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import evaluate
import importlib
import sys

sys.path.append(str(BASE_PATH / "data_baseline"))

# Import de vos utilitaires fixes
from data_utils import load_id2emb, load_descriptions_from_graphs, PreprocessedGraphDataset, collate_fn

# ==============================================================================
# 1. IMPORT DYNAMIQUE DU MODÈLE
# ==============================================================================
sys.path.append(str(BASE_PATH / "data_baseline"))
print(f"🔄 Chargement dynamique du module : {CODE_NAME}...")

try:
    module_code = importlib.import_module(CODE_NAME)
    MolGNN = module_code.MolGNN
    print("✅ Classe MolGNN importée avec succès !")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    raise e

# ==============================================================================
# 2. CONFIGURATION
# ==============================================================================
TRAIN_GRAPHS  = DATA_PATH / "train_graphs.pkl"
VAL_GRAPHS    = DATA_PATH / "validation_graphs.pkl"
TRAIN_EMB_CSV = DATA_PATH / "train_embeddings_ChemBERT.csv"
if not TRAIN_EMB_CSV.exists():
    TRAIN_EMB_CSV = DATA_PATH / "train_embeddings.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ Device: {DEVICE}")

# ==============================================================================
# 3. FONCTION DE PRÉDICTION
# ==============================================================================
@torch.no_grad()
def predict_val_retrieval():
    print("Chargement des embeddings texte...")
    train_emb = load_id2emb(str(TRAIN_EMB_CSV))
    train_ids = list(train_emb.keys())
    train_embs = torch.stack([train_emb[i] for i in train_ids]).to(DEVICE)
    train_embs = F.normalize(train_embs, dim=-1)

    train_id2desc = load_descriptions_from_graphs(str(TRAIN_GRAPHS))

    # Chargement Modèle
    emb_dim = train_embs.shape[1]
    model = MolGNN(out_dim=emb_dim).to(DEVICE)

    if not MODEL_PATH.exists():
        raise FileNotFoundError(f"Le fichier modèle n'existe pas : {MODEL_PATH}")

    state = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(state)
    model.eval()

    val_ds = PreprocessedGraphDataset(str(VAL_GRAPHS))
    val_dl = DataLoader(val_ds, batch_size=64, shuffle=False, collate_fn=collate_fn)

    with open(VAL_GRAPHS, "rb") as f:
        val_graphs_list = pickle.load(f)
    val_ref = {g.id: g.description for g in val_graphs_list}

    preds, refs = [], []
    offset = 0

    print("Inférence en cours...")
    for batch in val_dl:
        batch = batch.to(DEVICE)
        mol_emb = model(batch)
        mol_emb = F.normalize(mol_emb, dim=-1)

        sims = mol_emb @ train_embs.t()
        best = sims.argmax(dim=-1).cpu().tolist()

        bsz = batch.num_graphs
        batch_ids = val_ds.ids[offset:offset+bsz]
        offset += bsz

        for i, vid in enumerate(batch_ids):
            tid = train_ids[best[i]]
            preds.append(train_id2desc[tid])
            refs.append(val_ref[vid])

    return preds, refs

def compute_metrics(preds, refs):
    print("Calcul des métriques...")
    bleu_metric = evaluate.load("bleu")
    bleu_res = bleu_metric.compute(predictions=preds, references=[[r] for r in refs], max_order=4)
    bleu4 = bleu_res["bleu"]

    bert_metric = evaluate.load("bertscore")
    bert_res = bert_metric.compute(predictions=preds, references=refs, model_type="roberta-base", lang="en")
    bert_f1 = float(np.mean(bert_res["f1"]))

    bleu_norm = bleu4 if bleu4 <= 1.0 else bleu4 / 100.0
    final_proxy = 0.5 * bleu_norm + 0.5 * bert_f1
    return bleu4, bert_f1, final_proxy

# ==============================================================================
# 4. EXÉCUTION
# ==============================================================================
try:
    preds, refs = predict_val_retrieval()
    bleu4, bert_f1, final_proxy = compute_metrics(preds, refs)

    print("\n" + "="*40)
    print(f"RÉSULTATS POUR {CODE_NAME}")
    print("="*40)
    print(f"BLEU-4       : {bleu4:.5f}")
    print(f"BERTScore F1 : {bert_f1:.5f}")
    print(f"Final proxy  : {final_proxy:.5f}")
    print("-" * 40)
    print("Exemple de prédiction :")
    print(f"PRED: {preds[0][:150]}...")
    print(f"REF : {refs[0][:150]}...")

except Exception as e:
    print(f"\n❌ Une erreur est survenue : {e}")

🔄 Chargement dynamique du module : train_gps_chembert_mse...
✅ Classe MolGNN importée avec succès !
⚙️ Device: cuda
Chargement des embeddings texte...
Loading graphs from: /content/BornToOverfit/data_baseline/data/validation_graphs.pkl
Loaded 1000 graphs
Inférence en cours...
Calcul des métriques...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



RÉSULTATS POUR train_gps_chembert_mse
BLEU-4       : 0.10201
BERTScore F1 : 0.86330
Final proxy  : 0.48266
----------------------------------------
Exemple de prédiction :
PRED: The molecule is an oxysterol that is the 22S-hydroxy derivative of cholesterol. It is a 22-hydroxy steroid, an oxysterol and a 3beta-hydroxy-Delta(5)-...
REF : The molecule is an amino tetrasaccharide comprised of an L-iduronic acid residue, sulfated on O-2, a glucuronic acid residue, and two N-acetylated gal...
